In [1]:
import torchaudio
import torch 
import pandas as pd
import numpy as np
import re 
import os 
from torch.nn import functional as F
import random
import math
import pickle
import gc
import torch.nn as nn
from tqdm.notebook import tqdm
import os
import glob
import warnings
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings(action='ignore')

# residual connection
from res_stack import ResStack

In [2]:
base_dir = '/home/work/'
np1 = glob.glob(base_dir+'mel_dataset1/*.npy')
np2 = glob.glob(base_dir+'mel_dataset2/*.npy')
np3 = glob.glob(base_dir+'mel_dataset3/*.npy')
np4 = glob.glob(base_dir+'mel_dataset4/*.npy')
np5 = glob.glob(base_dir+'mel_dataset5/*.npy')
np6 = glob.glob(base_dir+'mel_dataset6/*.npy')
np7 = glob.glob(base_dir+'mel_dataset7/*.npy')
np8 = glob.glob(base_dir+'mel_dataset8/*.npy')

In [3]:
total = np1+np2+np3+np4+np5+np6+np7+np8 

In [4]:
train_csv = pd.read_csv('new_train.csv')
valid_csv = pd.read_csv('new_valid.csv')
all_csv =pd.read_csv('new_all.csv')

In [5]:
# def npy_list(csv):
#     npy_list = []
#     for song_path in tqdm(csv['npy_path']):
#         song_npy = np.load(song_path).squeeze()
#         npy_list.append(song_npy)
#     return npy_list

# train_list = npy_list(train_csv)
# valid_list = npy_list(valid_csv)

In [5]:
train_list = [np.load(song_path).squeeze() for song_path in tqdm(train_csv['npy_path'])]

  0%|          | 0/3825 [00:00<?, ?it/s]

In [6]:
valid_list = [np.load(song_path).squeeze() for song_path in tqdm(valid_csv['npy_path'])]

  0%|          | 0/675 [00:00<?, ?it/s]

In [7]:
all_list = train_list + valid_list

In [8]:
epochs = 150
batch_size = 32

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)

cuda


In [10]:
train_list[0].shape, valid_list[0].shape

((48, 7501), (48, 7501))

In [11]:
# epoch문 안에 들어가야되나?
#train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
#val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

In [17]:
# denoise encoding
def add_noise(data):
    noise = torch.randn(data.size()) * 0.2 
    # 무작위 작음은 torch.randn() 함수로 만들고 img.size()를 넣어 이미지와 같은 크기의 잡음을 만듭니다.
    # 잡음의 강도는 임의로 0.2로 정했습니다.
    noisy_data = data + noise
    return noisy_data

In [18]:
def train(model, train_loader):
    model.train()
    train_loss = 0
    for batch in train_loader: # 배치파일 다 만들면 train_loader [] 삭제하기
        
        #mel = np.load(batch_name) 
        batch = add_noise(batch) # denoise encoding
        mel = torch.FloatTensor(batch).to(DEVICE)
        
        optimizer.zero_grad()
        
        encode, output = model(mel)
        
        loss = criterion(output, mel)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
    train_loss /= (len(train_loader))

    return train_loss

def val(model, train_loader):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in train_loader:# 배치파일 다 만들면 train_loader [] 삭제하기
            
            #mel = np.load(batch_name)
            mel = torch.FloatTensor(batch).to(DEVICE)

            encode, output = model(mel)

            loss = criterion(output, mel)

            val_loss += loss.item()

    val_loss /= (len(train_loader))

    return val_loss


def get_mel_embeding(model, train_loader):
    model.eval()
    mel_embeding_li = []
    with torch.no_grad():
        for batch in train_loader: # 배치파일 다 만들면 train_loader [] 삭제하기
            
            #mel = np.load(batch_name)
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)
            mel_embeding_li.append(encode.detach().cpu().numpy())

    return mel_embeding_li

In [103]:
'''# Attention - 수정이 필요함... 정신이 혼미해
from torch.nn.parameter import Parameter

class channel_attention(nn.Module):    
    def __init__(self, channel, reduction=4):
        super(channel_attention, self).__init__()
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x): 
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        
        return x * y.expand_as(x)

class spatial_attention(nn.Module):   
    def __init__(self, channel, k_size=3):
        super(spatial_attention, self).__init__()
        self.conv = nn.Conv2d(channel, 1, kernel_size=k_size, padding='same', bias=False)
        self.sigmoid = nn.Sigmoid()
            
    def forward(self, x):
        y = self.conv(x)
        y = self.sigmoid(y)
    
        return x * y.expand_as(x)
    
    
class attention_module(nn.Module):
    def __init__(self, channel, attention_type=None):
        super(attention_module, self).__init__()
        self.CA = channel_attention(channel)
        self.SA = spatial_attention(channel)
        self.SC = nn.Sequential(
            channel_attention(channel),
            spatial_attention(channel),
        )
        self.SS = nn.Sequential(
            spatial_attention(channel),
            channel_attention(channel),
        )
        self.attention_type = attention_type
        
    def forward(self, x):
        if self.attention_type == 'channel':
            output = self.CA(x)
            
        elif self.attention_type == 'spatial':
            output = self.SA(x)
            
        elif self.attention_type == 'parellel_add':
            output1 = self.CA(x)
            output2 = self.SA(x)
            output = output1 + output2
            
        elif self.attention_type == 'parellel_mul':
            output1 = self.CA(x)
            output2 = self.SA(x)
            output = output1 * output2
            
        elif self.attention_type == 'serial_ca':
            output = self.SC(x)
            
        elif self.attention_type == 'serial_sa':
            output = self.SS(x)

        else:
            output  = x
    
        return output
'''

In [104]:
'''
# encoder <- 과거의 데이터를 바탕으로 미래를 예측 (t-1의 인과성 학습)
# decoder <- 미래의 데이터를 바탕으로 과거를 예측 (t+1의 인과성 학습)
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            channel_attention(512), 
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            channel_attention(256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            channel_attention(128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            channel_attention(64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            channel_attention(32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            channel_attention(16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            channel_attention(8),
            nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 128),
            nn.BatchNorm1d(128),
            nn.Tanh(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(128, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            channel_attention(16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            channel_attention(32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            channel_attention(64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            channel_attention(128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            channel_attention(256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            channel_attention(512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 512, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x = self.conv1(x)
        #print(x.shape)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.conv2(x)
        #print(x.shape)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.conv3(x)
        #print(x.shape)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.conv4(x)
        #print(x.shape)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.conv5(x)
        #print(x.shape)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.conv6(x)
        #print(x.shape)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.conv7(x)
        #print(x.shape)
        encode = self.encoder_fc(x.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.t_conv1(x)
        #print(x.shape)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv2(x)
        #print(x.shape)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv3(x)
        #print(x.shape)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        #print(x.shape)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv5(x)
        #print(x.shape)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv6(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv7(x)
        #print(x.shape)
        x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x
'''

In [19]:
# ResStack 추가
# LeakyReLU로 변경
# encoder <- 과거의 데이터를 바탕으로 미래를 예측 (t-1의 인과성 학습)
# decoder <- 미래의 데이터를 바탕으로 과거를 예측 (t+1의 인과성 학습)
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(), 
        )
        ResStack(512), 

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
        )
        ResStack(256),
        
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        ResStack(128),
        
        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
        )
        ResStack(64),
        
        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),
        )
        ResStack(32),
        
        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),
        )
        ResStack(16),
        
        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            nn.BatchNorm1d(8),
            nn.LeakyReLU(),
        )
        ResStack(8),
        
        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 128),
            nn.BatchNorm1d(128),
            nn.Tanh(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(128, 8 * 7501),
            nn.LeakyReLU(),
        )
        
        ResStack(8),
        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),
        )

        ResStack(16),
        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),
        )

        ResStack(32),
        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
        )

        ResStack(64),
        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )

        ResStack(128),
        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
        )

        ResStack(256),
        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
        )

        ResStack(512),
        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 512, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x = self.conv1(x)
        #print(x.shape)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.conv2(x)
        #print(x.shape)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.conv3(x)
        #print(x.shape)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.conv4(x)
        #print(x.shape)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.conv5(x)
        #print(x.shape)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.conv6(x)
        #print(x.shape)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.conv7(x)
        #print(x.shape)
        encode = self.encoder_fc(x.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.t_conv1(x)
        #print(x.shape)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv2(x)
        #print(x.shape)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv3(x)
        #print(x.shape)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        #print(x.shape)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv5(x)
        #print(x.shape)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv6(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv7(x)
        #print(x.shape)
        x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x

In [20]:
model = TimeAutoEncoder().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.L1Loss() # MSE에서 MAE로 바꿈

In [21]:
# train_list_shrt = [i[:,-3752:] for i in train_list]
# valid_list_shrt = [i[:,-3752:] for i in valid_list]

# train_list_shrt[0].shape

In [22]:
import time

epochs = 50

model_dir = '/home/work/Tcae_apply/model_dir2/'
min_loss = 987654321

#train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
#val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

for epoch in tqdm(range(1, epochs + 1)):
    start = time.time()
    
    train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
    val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

    train_loss = train(model = model, train_loader = train_batch_li) 
    val_loss = val(model = model, train_loader = val_batch_li) 
    end = time.time()

    print(f'EPOCH:{epoch}, Train Loss:{train_loss}, Val Loss:{val_loss}, 학습 시간: {end - start}')
    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), model_dir + f'TimeAutoEncoder_val.pt')
        print('모델 저장')
        
    del train_batch_li
    del val_batch_li
 # Resstack

  0%|          | 0/50 [00:00<?, ?it/s]

EPOCH:1, Train Loss:20.331643921988352, Val Loss:19.834490004039946, 학습 시간: 36.404571771621704
모델 저장
EPOCH:2, Train Loss:20.26914045189609, Val Loss:19.88207163129534, 학습 시간: 36.76791453361511
EPOCH:3, Train Loss:20.232004197705695, Val Loss:19.796177046639578, 학습 시간: 36.79360914230347
모델 저장
EPOCH:4, Train Loss:20.18888117686039, Val Loss:19.7868595123291, 학습 시간: 36.99477672576904
모델 저장
EPOCH:5, Train Loss:20.157301918799135, Val Loss:19.66015238988967, 학습 시간: 36.9385769367218
모델 저장
EPOCH:6, Train Loss:20.099749981856146, Val Loss:19.64265841529483, 학습 시간: 36.753530502319336
모델 저장
EPOCH:7, Train Loss:20.009544428657083, Val Loss:19.642210097539994, 학습 시간: 36.98733329772949
모델 저장
EPOCH:8, Train Loss:19.89348023679076, Val Loss:19.571494602021716, 학습 시간: 37.03239560127258
모델 저장
EPOCH:9, Train Loss:19.82877737734498, Val Loss:19.664703051249187, 학습 시간: 38.1425244808197
EPOCH:10, Train Loss:19.704275379661752, Val Loss:19.58768131619408, 학습 시간: 37.34929633140564
EPOCH:11, Train Loss:19.573

In [90]:
'''
import time

model_dir = '/home/work/Tcae_apply/model_dir/'
min_loss = 987654321

train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

for epoch in tqdm(range(1, epochs + 1)):
    start = time.time()
    
    #train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
    #val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

    train_loss = train(model = model, train_loader = train_batch_li) 
    val_loss = val(model = model, train_loader = val_batch_li) 
    end = time.time()

    print(f'EPOCH:{epoch}, Train Loss:{train_loss}, Val Loss:{val_loss}, 학습 시간: {end - start}')
    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), model_dir + f'TimeAutoEncoder_val.pt')
        print('모델 저장')
        
    #del train_batch_li
    #del valid_batch_li
# leaky 
'''

  0%|          | 0/150 [00:00<?, ?it/s]

EPOCH:1, Train Loss:20.34235031865224, Val Loss:19.95621054513114, 학습 시간: 32.697444677352905
모델 저장
EPOCH:2, Train Loss:20.26667368912897, Val Loss:19.857975823538645, 학습 시간: 32.84595990180969
모델 저장
EPOCH:3, Train Loss:20.256751228781308, Val Loss:19.868038813273113, 학습 시간: 33.06649208068848
EPOCH:4, Train Loss:20.216806211391418, Val Loss:19.84003457568941, 학습 시간: 33.23177766799927
모델 저장


KeyboardInterrupt: 

In [ ]:
#model = TimeAutoEncoder().to(DEVICE)
#model.load_state_dict(torch.load(model_dir + f'TimeAutoEncoder_val.pt', map_location = DEVICE))

In [ ]:
#mel_embeding_li = get_mel_embeding(model = model, train_loader = test_batch_li)
#mel_embeding = np.concatenate(mel_embeding_li, axis = 0)

In [ ]:
#np.save(batch_data_dir + 'mel_embeding_val.npy', mel_embeding)